In [1]:
import numpy as np
import pandas as pd
import datadotworld as dw
from datetime import datetime

In [5]:
# 0
        # Without authentication.
        # data = pd.read_csv("https://query.data.world/s/nltr6xsnpwrhcfx4dacus3gc73kmbz")
        # data.info()

query = dw.query(
	'siyeh/austin-animal-center-live-data', 
    'SELECT * FROM austin_animal_center_intakes')
data = query.dataframe
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133515 entries, 0 to 133514
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   animal_id         133515 non-null  object        
 1   name              92905 non-null   object        
 2   datetime          133515 non-null  datetime64[ns]
 3   monthyear         133515 non-null  datetime64[ns]
 4   found_location    133515 non-null  object        
 5   intake_type       133515 non-null  object        
 6   intake_condition  133515 non-null  object        
 7   animal_type       133515 non-null  object        
 8   sex_upon_intake   133515 non-null  object        
 9   age_upon_intake   133515 non-null  object        
 10  breed             133515 non-null  object        
 11  color             133515 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 12.2+ MB


<b>REMOVE duplicate rows, FILL null values</b>

In [38]:
# 1
data.drop_duplicates(inplace=True)

In [39]:
# 2
data.drop("monthyear", axis=1, inplace=True)

In [40]:
# 3
data["name"].fillna("Unknown", inplace=True)
data["name"] = data["name"].str.replace("*", "", regex=False)
data.loc[data["name"]=="", "name"] = "Unknown"

In [41]:
# 4
data["sex_upon_intake"].fillna("Unknown", inplace=True)

<b>ADJUST & CREATE additional COLUMNS</b>

In [42]:
# 5
data.loc[data["name"]==data["animal_id"], "name"] = "Unknown"

In [43]:
# 6
data["in_time2"] = np.where(data.datetime.dt.hour < 12, "AM", "PM")
data["datetime"] = data["datetime"].astype("str")
data[["in_date", "in_time1"]] = data["datetime"].str.split(pat=" ", n=1, expand=True)

In [46]:
# 7
data["in_date"] = pd.to_datetime(data["in_date"], format="%Y-%m-%d")#.dt.strftime("%d/%m/%Y")
#data["in_date"] = pd.to_datetime(data["in_date"], format="%d/%m/%Y")
data.drop("datetime", axis=1, inplace=True)

In [48]:
# 8
data.rename(columns={"intake_Type":"in_type",
                     "intake_condition":"in_condition",
                     "sex_upon_intake":"in_sex",
                     "age_upon_intake":"in_age"}, inplace=True)

In [49]:
# 9
data[["aui1", "aui2"]] = data["in_age"].str.split(n=1, expand=True)
data["aui1"] = data["aui1"].astype("int")
data.loc[data["aui1"]<0, "aui1"] = 0

In [50]:
# 10
conditions = [(data["aui2"]=="day") | (data["aui2"]=="days"),
              (data["aui2"]=="week") | (data["aui2"]=="weeks"),
              (data["aui2"]=="month") | (data["aui2"]=="months"),
              (data["aui2"]=="year") | (data["aui2"]=="years")]
values = [1/30, 7/30, 1, 12]
data["aui3"] = np.select(conditions, values)

In [51]:
# 11
data["in_age2"] = data["aui1"] * data["aui3"]
data.drop(["aui1", "aui2", "aui3"], axis=1, inplace=True)

In [52]:
# 12
data.sort_values(by="in_date", inplace=True)

<b>EXPORT period for AnimalCenter-Calendar</b>

In [ ]:
start = data.head(1)["in_date"].copy()
start = pd.to_datetime(start, format="%Y/%m/%d").dt.strftime("%m/%d/%Y").values[0]
start

'10/01/2013'

In [ ]:
end = data.tail(1)["in_date"].copy()
end = pd.to_datetime(end, format="%Y/%m/%d").dt.strftime("%m/%d/%Y").values[0]
end

'10/12/2021'

<b>RECHECK data</b>

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132284 entries, 103868 to 129509
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   animal_id       132284 non-null  object        
 1   name            132284 non-null  object        
 2   found_location  132284 non-null  object        
 3   in_type         132284 non-null  object        
 4   in_condition    132284 non-null  object        
 5   animal_type     132284 non-null  object        
 6   in_sex          132284 non-null  object        
 7   in_age          132284 non-null  object        
 8   breed           132284 non-null  object        
 9   color           132284 non-null  object        
 10  in_date         132284 non-null  datetime64[ns]
 11  in_time1        132284 non-null  object        
 12  in_time2        132284 non-null  object        
 13  in_age2         132284 non-null  float64       
dtypes: datetime64[ns](1), float64(1

In [ ]:
data.sort_values(by="in_date", ascending=False)

,animal_id,name,found_location,in_type,in_condition,animal_type,in_sex,in_age,breed,color,in_date,in_time1,in_time2,in_age2
129509,A844094,Unknown,7201 Levander Loop in Austin (TX),Abandoned,Normal,Dog,Intact Male,1 month,Labrador Retriever Mix,Brown/Black,2021-10-12,10:53:00,AM,1.000000
129573,A844111,Unknown,1111 Chicon Street in Austin (TX),Stray,Injured,Cat,Intact Male,1 month,Domestic Shorthair,Brown Tabby/White,2021-10-12,01:25:00,PM,1.000000
129568,A844103,Unknown,1104 Walthon Lane in Austin (TX),Stray,Normal,Cat,Intact Male,2 weeks,Domestic Shorthair Mix,Black,2021-10-12,12:23:00,PM,0.466667
129566,A844097,Mila,Austin (TX),Owner Surrender,Normal,Dog,Intact Female,1 year,Pit Bull,Tan,2021-10-12,12:38:00,PM,12.000000
129618,A844123,Unknown,11408 Moored Rd in Austin (TX),Stray,Injured,Cat,Intact Female,1 month,Domestic Shorthair,Black/White,2021-10-12,03:13:00,PM,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27394,A639030,Cate,46Th And Ave G in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,German Shepherd Mix,Tan,2013-10-01,01:36:00,PM,84.000000
69140,A664305,Annabelle,12315 Rayden St. in Travis (TX),Stray,Normal,Dog,Intact Female,1 month,Anatol Shepherd/Labrador Retriever,White,2013-10-01,04:10:00,PM,1.000000
39713,A664267,Asti,12501 Tech Ridge Blvd in Austin (TX),Stray,Normal,Dog,Intact Female,4 months,Pit Bull Mix,White/Black,2013-10-01,11:59:00,AM,4.000000
37044,A656387,Ned,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,2 years,Labrador Retriever/Chow Chow,Red/White,2013-10-01,07:05:00,PM,24.000000


In [ ]:
#data.groupby("animal_id").count().sort_values(by="name").tail(50)

In [ ]:
#data[data["animal_id"]=="A700407"].sort_values(by="in_date")

<b>EXPORT data</b>

In [ ]:
# 13
data.to_csv("preprocessed_AnimalCenter-AustinTX_Intakes.csv", index=False)